In [1]:
import win32com.client
import pandas as pd
import numpy as np
import datetime as dt
# from glob import glob
import time

In [2]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
print(instCpCybos.IsConnect)

1


In [3]:
# 증권시장에서 상장되어 있는 종목의 갯수 출력 -> 새로운 종목이 상장되거나 폐지됨에 따라 값이 달라진다.
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(instCpStockCode.GetCount())

3963


In [4]:
# 0 -> 종목코드 : 요청할 종목의 종목 코드
# 1 -> 요청구분 : 1은 기간으로 요청, 2는 갯수로 요청
# 2 -> 요청 종료일 : YYYYMMDD 형식
# 3 -> 요청 시작일 : YYYYMMDD 형식
# 4 -> 요청개수 : 요청할 데이터의 개수
# 5 -> 필드 = 0: 날짜, 1:시간, 2:시가, 3: 고가, 4: 저가, 5: 종가, 6: 전일대비, 8: 거래량, 9: 거래대금, 10: 누적체결매도수량
#             11: 누적체결매수수량, 12: 상장주식수, 13: 시가총액, 14: 외국인주문한도수량, 15: 외국인주문가능수량, 16: 외국인현보유수량
#             17: 외국인현보유비율, 18: 수정주가일자, 19: 수정주가비율, 20: 기관순매수, 21: 기관누적순매수, 22: 등락주선, 23: 등락비율
#             24: 예탁금, 25: 주식회전율, 26: 거래성립률, 27: 대비부호
# 6 -> 차트 구분 = D: 일, W: 주, m: 분, T: 틱
# 9 -> 수정주가 = 0: 무수정주가, 1: 수정주가

## 데이터 수집

In [5]:
code = [] # 종목코드
name = [] # 종목명

for i in range(0, instCpStockCode.GetCount()):
    code.append(instCpStockCode.GetData(0, i))
    name.append(instCpStockCode.GetData(1, i))

codeNameList = pd.DataFrame({
      '종목코드':code
    , '종목명':name
})
codeNameList

,종목코드,종목명
0,A000020,동화약품
1,A000040,KR모터스
2,A000050,경방
3,A000070,삼양홀딩스
4,A000075,삼양홀딩스우
...,...,...
3958,J52J712,미래J712케이티지콜
3959,J52J713,미래J713케이티지콜
3960,J52J714,미래J714케이티지콜
3961,J52J715,미래J715현대상선콜


In [6]:
codeNameList[codeNameList['종목명'] == '에스에너지']

,종목코드,종목명
1600,A095910,에스에너지


In [7]:
stock_list = ['셀바스AI', '투비소프트', '플리토','뷰노','알체라']
hj_stock_list = codeNameList[codeNameList['종목명'].isin(stock_list)]
hj_stock_list

,종목코드,종목명
1421,A079970,투비소프트
1721,A108860,셀바스AI
2763,A300080,플리토
2958,A338220,뷰노
2979,A347860,알체라


In [8]:
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

In [9]:
def getjusik(jusikCode, jusikName, start, end, dataDf):
    code = [] # 종목코드
    name = [] # 종목명
    c0 = []   # 날짜
    c1 = []   # 시간
    c2 = []   # 시가
    c3 = []   # 고가
    c4 = []   # 저가
    c5 = []   # 종가
    c6 = []   # 전일대비
    c8 = []   # 거래량
    c9 = []   # 거래대금
    c10 = []  # 누적체결매도수량
    c11 = []  # 누적체결매수수량
    c12 = []  # 상장주식수
    c13 = []  # 시가총액
    c14 = []  # 외국인주문한도수량
    c15 = []  # 외국인주문가능수량
    c16 = []  # 외국인현보유수량
    c17 = []  # 외국인현보유비율
    c18 = []  # 수정주가일자
    c19 = []  # 수정주가비율
    c20 = []  # 기관순매수량
    c21 = []  # 기관누적순매수량
    c22 = []  # 등락주선
    c23 = []  # 등락비율
    c24 = []  # 예탁금
    c25 = []  # 주식회전율

    instStockChart.SetInputValue(0, jusikCode)
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(2, end)
    instStockChart.SetInputValue(3, start)
    instStockChart.SetInputValue(4, 2000)
    instStockChart.SetInputValue(5, (0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25))
    instStockChart.SetInputValue(6, ord('m'))
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest()

    numData = instStockChart.GetHeaderValue(3)
                                                            
    for i in range(numData):
        code.append(jusikCode)
        name.append(jusikName)
        c0.append(str(instStockChart.GetDataValue(0, i)))
        c1.append(str(instStockChart.GetDataValue(1, i)))
        c2.append(instStockChart.GetDataValue(2, i))
        c3.append(instStockChart.GetDataValue(3, i))
        c4.append(instStockChart.GetDataValue(4, i))
        c5.append(instStockChart.GetDataValue(5, i))
        c6.append(instStockChart.GetDataValue(6, i))
        c8.append(instStockChart.GetDataValue(7, i))
        c9.append(instStockChart.GetDataValue(8, i))
        c10.append(instStockChart.GetDataValue(9, i))
        c11.append(instStockChart.GetDataValue(10, i))
        c12.append(instStockChart.GetDataValue(11, i))
        c13.append(instStockChart.GetDataValue(12, i))
        c14.append(instStockChart.GetDataValue(13, i))
        c15.append(instStockChart.GetDataValue(14, i))
        c16.append(instStockChart.GetDataValue(15, i))
        c17.append(instStockChart.GetDataValue(16, i))
        c18.append(str(instStockChart.GetDataValue(17, i)))
        c19.append(instStockChart.GetDataValue(18, i))
        c20.append(instStockChart.GetDataValue(19, i))
        c21.append(instStockChart.GetDataValue(20, i))
        c22.append(instStockChart.GetDataValue(21, i))
        c23.append(instStockChart.GetDataValue(22, i))
        c24.append(instStockChart.GetDataValue(23, i))
        c25.append(instStockChart.GetDataValue(24, i))

    dataDf['종목코드'] = code
    dataDf['종목명'] = name
    dataDf['날짜'] = c0
    dataDf['시간'] = c1
    dataDf['시가'] = c2
    dataDf['고가'] = c3
    dataDf['저가'] = c4
    dataDf['종가'] = c5
    dataDf['전일대비'] = c6
    dataDf['거래량'] = c8
    dataDf['거래대금'] = c9
    dataDf['누적체결매도수량'] = c10
    dataDf['누적체결매수수량'] = c11
    dataDf['상장주식수'] = c12
    dataDf['시가총액'] = c13
    dataDf['외국인주문한도수량'] = c14
    dataDf['외국인주문가능수량'] = c15
    dataDf['외국인현보유수량'] = c16
    dataDf['외국인현보유비율'] = c17
    dataDf['수정주가일자'] = c18
    dataDf['수정주가비율'] = c19
    dataDf['기관순매수량'] = c20
    dataDf['기관누적순매수량'] = c21
    dataDf['등락주선'] = c22
    dataDf['등락비율'] = c23
    dataDf['예탁금'] = c24
    dataDf['주식회전율'] = c25

In [11]:
a = dt.datetime(2023,6,21)
b = dt.datetime.now()
#a = dt.datetime(2022,1,1)
#b = dt.datetime(2022,12,31)
number = b-a
stock_list = [1421,1721,2763,2958,2979]

for j in stock_list:
    frame = []
    for i in range(0, number.days, 2):
        dataDf = pd.DataFrame()
        c = a + dt.timedelta(days=i)
        d = a + dt.timedelta(days=i+1)
        getjusik(codeNameList['종목코드'].iloc[j], codeNameList['종목명'].iloc[j], c.strftime('%Y%m%d'), d.strftime('%Y%m%d'), dataDf)
        frame.append(dataDf)
        time.sleep(0.3)
    hapdata = pd.concat(frame)
    hapdata.to_csv('./'+codeNameList['종목명'].iloc[j]+' 주식 데이터.csv', encoding='euc-kr', index=False)
    print(codeNameList['종목명'].iloc[j]+' 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!')

print('수신요청 종료')
# frame

투비소프트 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
셀바스AI 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
플리토 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
뷰노 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
알체라 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
수신요청 종료
